## Create FIPS csv

* Creates a csv file of state and county FIPS codes for each location, defined by coordinate pairs. This works by creating a point feature class (using the ArcPy module) and doing a spatial join with a feature class of counties. 
* Requires*: ArcPy

In [1]:
#Import libraries
import sys, os, csv
import arcpy
import numpy as np

In [2]:
#Enable overwriting of output (for debugging)
arcpy.env.overwriteOutput = True

In [3]:
#Get the county shapefile in the Data dir
countyFC = "../../Data/cb_2016_us_county_500k.shp"

In [4]:
#Get the csv file with coordinates
csvFile = "../../Data/Year2000.csv"

In [5]:
#Create an XY Event layer from the CSV file
xyEventLayer = arcpy.MakeXYEventLayer_management(csvFile,"Longitude","Latitude","XY_Lyr")

In [6]:
#Create a feature class from the event layer
xyFeatureClass = arcpy.CopyFeatures_management(xyEventLayer,"in_memory/XYFeatures")

In [7]:
#Spatial join with countyFC (this takes a moment...)
outJoinFC = "in_memory/JoinFC"
joinFC = arcpy.SpatialJoin_analysis(xyFeatureClass,countyFC,outJoinFC)

In [8]:
#Convert table to numpy array (Set null values to -1)
nullDict = {'GEOID':-1,'STATEFP':-1}
np1 = arcpy.da.TableToNumPyArray(joinFC,['GEOID','STATEFP'],null_value=nullDict)

In [9]:
#Combine the GEOID and STATEFP into a 2d array
npX = np.vstack([np1["GEOID"], np1["STATEFP"]])

In [10]:
#Write to a file
np.savetxt("../../Data/FIPS.csv",
           npX.T,delimiter=',',
           fmt='%s',
           header=u"COFIPS, STATEFIPS")